In [1]:
import requests as req
from bs4 import BeautifulSoup

import pandas as pd
import pylab as pl
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium.plugins import MarkerCluster

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


Solving environment: done

# All requested packages already installed.



In [9]:
resp = req.get("https://en.wikipedia.org/wiki/Postal_codes_in_Singapore")

In [10]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [11]:
table = soup.find('table', attrs={'class':'wikitable'})
#table

In [12]:
data=[]
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
#data

In [13]:
del data[0]
#data

df=pd.DataFrame(list(data), columns=['District','Postal Code','Area'])  
df.head()

,District,Postal Code,Area
0,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park"
1,02,"07, 08","Anson, Tanjong Pagar"
2,03,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru"
3,04,"09, 10","Telok Blangah, Harbourfront"
4,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."


In [14]:
df_sg = pd.read_csv('sg.csv')
df_sg=df.join(df_sg.set_index('Area'), on='Area')
df_sg

,District,Postal Code,Area,Latitude,Longitude
0,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park",1.281632,103.857248
1,02,"07, 08","Anson, Tanjong Pagar",1.272167,103.842780
2,03,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru",1.286189,103.817772
3,04,"09, 10","Telok Blangah, Harbourfront",1.271282,103.817022
4,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New...",1.295909,103.777698
5,06,17,"High Street, Beach Road (part)",1.292577,103.849194
6,07,"18, 19","Middle Road, Golden Mile",1.300595,103.858292
7,08,"20, 21","Little India, Farrer Park, Jalan Besar, Lavender",1.310853,103.854433
8,09,"22, 23","Orchard, Cairnhill, River Valley",1.302264,103.839234
9,10,"24, 25, 26, 27","Ardmore, Bukit Timah, Holland Road, Tanglin",1.313139,103.806978


In [15]:
CLIENT_ID = 'RTZWY2Y3T1EVLYSCHAIHW4N5W5SGR21LOG1DJFIKWPQBXSZI' # your Foursquare ID
CLIENT_SECRET = 'A0CY0JWXE3MZV3IQX4AHQH1MICJI3X4D4UFSZC21UY5VF3DX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = req.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
singapore_venues = getNearbyVenues(names=df_sg['Area'],latitudes=df_sg['Latitude'],longitudes=df_sg['Longitude'])

Raffles Place, Cecil, Marina, People's Park
Anson, Tanjong Pagar
Bukit Merah, Queenstown, Tiong Bahru
Telok Blangah, Harbourfront
Pasir Panjang, Hong Leong Garden, Clementi New Town
High Street, Beach Road (part)
Middle Road, Golden Mile
Little India, Farrer Park, Jalan Besar, Lavender
Orchard, Cairnhill, River Valley
Ardmore, Bukit Timah, Holland Road, Tanglin
Watten Estate, Novena, Thomson
Balestier, Toa Payoh, Serangoon
Macpherson, Braddell
Geylang, Eunos
Katong, Joo Chiat, Amber Road
Bedok, Upper East Coast, Eastwood, Kew Drive
Loyang, Changi
Simei, Tampines, Pasir Ris
Serangoon Garden, Hougang, Punggol
Bishan, Ang Mo Kio
Upper Bukit Timah, Clementi Park, Ulu Pandan
Jurong, Tuas
Hillview, Dairy Farm, Bukit Panjang, Choa Chu Kang
Lim Chu Kang, Tengah
Kranji, Woodgrove, Woodlands
Upper Thomson, Springleaf
Yishun, Sembawang
Seletar


In [20]:
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")
singapore_onehot['Area'] = singapore_venues['Area'] 
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]
singapore_grouped = singapore_onehot.groupby('Area').mean().reset_index()
singapore_grouped_jp = singapore_grouped[['Area','Japanese Restaurant']]

In [21]:
singapore_grouped_jp.columns = ['Area', 'JapaneseResto']
singapore_grouped_jp = singapore_grouped_jp.drop(singapore_grouped_jp[singapore_grouped_jp.JapaneseResto == 0].index)

In [22]:
singapore_merged = singapore_grouped_jp.join(df_sg.set_index('Area'), on='Area')
singapore_merged.sort_values(by=['JapaneseResto'],ascending=False)

,Area,JapaneseResto,District,Postal Code,Latitude,Longitude
26,"Watten Estate, Novena, Thomson",0.250000,11,"28, 29, 30",1.333708,103.819433
17,"Orchard, Cairnhill, River Valley",0.110000,09,"22, 23",1.302264,103.839234
0,"Anson, Tanjong Pagar",0.104478,02,"07, 08",1.272167,103.842780
6,"Geylang, Eunos",0.100000,14,"38, 39, 40, 41",1.324518,103.899600
7,"High Street, Beach Road (part)",0.084507,06,17,1.292577,103.849194
19,"Raffles Place, Cecil, Marina, People's Park",0.052632,01,"01, 02, 03, 04, 05, 06",1.281632,103.857248
10,"Katong, Joo Chiat, Amber Road",0.029412,15,"42, 43, 44, 45",1.303239,103.899318
16,"Middle Road, Golden Mile",0.020000,07,"18, 19",1.300595,103.858292


In [25]:
colors_array = cm.rainbow(np.linspace(0, 1, 9))
rainbow = [colors.rgb2hex(i) for i in colors_array]

sg_geo = r'sg.json'
sg_map = folium.Map(location=[1.3521,103.8198], zoom_start=12, tiles='Stamen Terrain')

sg_map.choropleth(
    geo_data=sg_geo,
    data=singapore_merged,
    columns=['Area','JapaneseResto'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Singapore'
)

markers_colors = []
for lat, lon, poi in zip(singapore_merged['Latitude'], singapore_merged['Longitude'], singapore_merged['Area']):
    label = folium.Popup('Area ' + str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(sg_map)
       
sg_map

In [26]:
best_jp=singapore_merged.sort_values(by=['JapaneseResto'],ascending=False).head(3)
best_jp

,Area,JapaneseResto,District,Postal Code,Latitude,Longitude
26,"Watten Estate, Novena, Thomson",0.250000,11,"28, 29, 30",1.333708,103.819433
17,"Orchard, Cairnhill, River Valley",0.110000,09,"22, 23",1.302264,103.839234
0,"Anson, Tanjong Pagar",0.104478,02,"07, 08",1.272167,103.842780


In [64]:
sg_geo = r'sg.json'   
radius = 500
resto_results=''

for lat, lon, poi in zip(best_jp['Latitude'], best_jp['Longitude'], best_jp['Area']):
    url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&radius={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, radius, lat, lon, VERSION)
    results = req.get(url).json()
    if len(results['response']['venues']) == 0:
        recommended_area='Not Available'
    else:
        trending_venues = results['response']['venues']
        trending_venues_df = json_normalize(trending_venues)
        # filter columns
        columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
        trending_venues_df = trending_venues_df.loc[:, columns_filtered]

        # filter the category for each row
        trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)
        LIMIT = 10
        search_query = 'Japanese Restaurant'
        radius = 500

        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

        resto_results = req.get(url).json()
        venues = resto_results['response']['venues']
        recommended_area=poi
        # tranform venues into a dataframe
        venue_df = json_normalize(venues)
        break

recommended_area='Orchard, Cairnhill, River Valley'
lat=1.302264
lon=103.839234

In [68]:
print('Suggested area to establish a new Japanese Restaurant based on trending location and popular location for Japanese Restaurant is '+recommended_area+'.')
#if len(resto_results['response']['venues']) == 0:
if recommended_area == '0':
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[1.3521,103.8198], zoom_start=12, tiles='Stamen Terrain')
    venues_map.choropleth(
        geo_data=sg_geo,
        data=singapore_merged,
        columns=['Area','JapaneseResto'],
        key_on='feature.properties.name',
        fill_color='YlOrRd', 
        fill_opacity=0.7, 
        line_opacity=0.2,
        legend_name='Singapore'
    )
    # add the trending venues as blue circle markers
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        poup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
    


Suggested area to establish a new Japanese Restaurant based on trending location and popular location for Japanese Restaurant is Orchard, Cairnhill, River Valley.


In [69]:
venues_map